# Day 11

Too much text.
[You can see it for yourself here.](https://adventofcode.com/2021/day/11)

TL;DR:
>You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.
> 
> There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.
> 
> Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus.

## Load n Read stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

In [ ]:
example_input = [
    '5483143223',
    '2745854711',
    '5264556173',
    '6141336146',
    '6357385478',
    '4167524645',
    '2176841721',
    '6882881134',
    '4846848554',
    '5283751526',
]

In [ ]:
with open('../data/day_11_puzzle_1') as f:
    input_data = f.readlines()

# input_data = example_input
input_data

## Clean your data maaaan

- Remove newline characters (`\n`) from your data
- Convert each row of data to a list of integers (`int`)
- Use hard to read list comprehension for bonus minus points 🌚

In [ ]:
grid = [[int(octopus_energy) for octopus_energy in row.replace('\n','')] for row in input_data]
grid = np.array(grid)
print('Your grid has a shape of:', grid.shape)
print('And it\' been initialized as:')
print(grid)

## Run octopus simulation

To configure the simulation, please input the number of `steps` you wish to simulate to.

The octopus simulator can be used in two ways:
1. Set the number of steps in the future that you need to predict to in order to get the exact number of octopus flashes.
2. Set the number of steps to a large number (e.g. `500`) in order to find when the octopuses first sync.
If you don't see any results keep increasing the simulation step number until you get a result!

In [ ]:
steps = 500

In [ ]:
live_grid = grid.copy()
grid_rows = live_grid.shape[0]
grid_cols = live_grid.shape[1]
flashes = 0
simultaneous_flash_flag = False
frames = [live_grid.tolist()]

for step in range(steps):
    live_grid += 1
    changes = True
    previous_grid_state = live_grid.copy()
    loops_to_changes = 0
    while changes:
        loops_to_changes += 1
        for row in range(grid_rows):
            for col in range(grid_cols):
                octopus = live_grid[row, col]
                if octopus > 9:
                    # octopus flashed!
                    live_grid[row, col] = 0
                    flashes += 1
                    # update octopuses above
                    if row - 1 >= 0:
                        if col + 1 <= grid_cols - 1:
                            if live_grid[row - 1, col + 1] != 0:
                                live_grid[row - 1, col + 1] += 1

                        if live_grid[row - 1, col] != 0:
                            live_grid[row - 1, col] += 1

                        if col - 1 >= 0:
                            if live_grid[row - 1, col - 1] != 0:
                                live_grid[row - 1, col - 1] += 1

                    # update octopuses below
                    if row + 1 <= grid_rows - 1:
                        if col + 1 <= grid_cols - 1:
                            if live_grid[row + 1, col + 1] != 0:
                                live_grid[row + 1, col + 1] += 1

                        if live_grid[row + 1, col] != 0:
                            live_grid[row + 1, col] += 1

                        if col - 1 >= 0:
                            if live_grid[row + 1, col - 1] != 0:
                                live_grid[row + 1, col - 1] += 1

                    # update octopuses left
                    if col - 1 >= 0:
                        if live_grid[row, col - 1] != 0:
                            live_grid[row, col - 1] += 1
                    # update octopuses right
                    if col + 1 <= grid_cols - 1:
                        if live_grid[row, col + 1] != 0:
                            live_grid[row, col + 1] += 1

        if (live_grid == previous_grid_state).all():
            changes = False
        previous_grid_state = live_grid.copy()

    frames.append(live_grid.tolist())

    if live_grid.sum() == 0:
        simultaneous_flash_flag = True
        print(f'In step {step + 1:5d} all octopuses will flash simultaneously!')

print(f'Octopuses flashed {flashes:6d} times')

if not simultaneous_flash_flag:
    print('Looks like the octopuses did not flash simultaneously in any future step of your simulation. Try increasing the number of steps!')
            

            

## Animate!

A blast from the past!
The great tutorial author Jake came to the rescue!
With his [tutorial](https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/) we managed to create a sweet animation of the octopus simulation!

Thus we modified his solution to fit our submarine system.

In [ ]:
"""
Matplotlib Animation Example

author: Jake Vanderplas
email: vanderplas@astro.washington.edu
website: http://jakevdp.github.com
license: BSD
Please feel free to use and modify this, but keep the above information. Thanks!
"""

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
img = plt.imshow(frames[0], interpolation='none')
# plt.colorbar(boundaries=BoundaryNorm([i for i in range(10)], 10))
plt.axis('off')
plt.title(f'Step {0:4d}')

# initialization function: plot the background of each frame
def init():
    img.set_data(frames[0])
    plt.title(f'Step {0:4d}')
    return img,

# animation function.  This is called sequentially
def animate(i):
    img.set_data(frames[i])
    plt.title(f'Step {i+1:4d}')
    return img,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=500, interval=1, blit=True)

# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
anim.save('outputs/day-11_octopuses_flashing.mp4', fps=10, extra_args=['-vcodec', 'libx264'])

plt.show()

# Tada!

Enjoy your sweet octopus simulation over at [day-11_octopuses_flashing.mp4](/solutions/outputs/day-11_octopuses_flashing.mp4)